In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
df_test=pd.read_csv('/kaggle/input/airbnb-listings-in-major-us-cities-deloitte-ml/test.csv')

In [ ]:
df_test

In [ ]:
train_df=pd.read_csv('/kaggle/input/airbnb-listings-in-major-us-cities-deloitte-ml/train.csv')

In [ ]:
train_df

In [ ]:
#drop unnecessary features from both train and test set

train_df=train_df.drop(['first_review','host_response_rate','review_scores_rating','neighbourhood','last_review','thumbnail_url','zipcode'],axis=1)

In [ ]:
df_test=df_test.drop(['first_review','host_response_rate','review_scores_rating','neighbourhood','last_review','thumbnail_url','zipcode'],axis=1)

In [ ]:
Id=df_test['id']

In [ ]:
train_df.corr()

In [ ]:
train_df=train_df.drop(['amenities','longitude','latitude','host_has_profile_pic','host_identity_verified','host_since','name','id'],axis=1)

In [ ]:
df_test=df_test.drop(['amenities','longitude','latitude','host_has_profile_pic','host_identity_verified','host_since','name','id'],axis=1)

In [ ]:
train_df=train_df.drop(['city','description'],axis=1)

In [ ]:
df_test=df_test.drop(['city','description'],axis=1)

In [ ]:
train_df

In [ ]:
train_df.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
#for train data
dict_1={}

dict_1=dict(train_df.isnull().sum())

In [ ]:
#for test_data
dict_2={}
dict_2=dict(df_test.isnull().sum())

In [ ]:
#aggregating all the nan values form both datasets form imputation
list_1=[]
list_2=[]
for key,values in dict_1.items():
    if train_df[key].dtype=='object' and values>0:
        list_1.append(key)
    if train_df[key].dtype!='object' and values>0:
        list_2.append(key)

In [ ]:
list_1,list_2

In [ ]:
list_3=[]
list_4=[]
for key,values in dict_2.items():
    if df_test[key].dtype=='object' and values>0:
        list_3.append(key)
    if df_test[key].dtype!='object' and values>0:
        list_4.append(key)

In [ ]:
list_2

In [ ]:
for i in list_2:
    train_df[i]=train_df[i].fillna(train_df[i].mode()[0])

In [ ]:
for i in list_2:
    train_df[i]=train_df[i].astype('int64')

In [ ]:
list_4

In [ ]:
for i in list_4:
    df_test[i]=df_test[i].fillna(df_test[i].mode()[0])

In [ ]:
for i in list_4:
    df_test[i]=df_test[i].astype('int64')

In [ ]:
train_df.isnull().sum()

In [ ]:
df_test.isnull().sum()

In [ ]:
y=train_df['log_price']

In [ ]:
train_df=train_df.drop(['log_price'],axis=1)

In [ ]:
#let separate the catrgorical features for encoding process
list_cate=[]
for i in list(train_df.columns):
    if train_df[i].dtype=='object':
        list_cate.append(i)
   

In [ ]:
test_cate=[]
for i in list(df_test.columns):
    if df_test[i].dtype=='object':
        test_cate.append(i)
    

In [ ]:
from sklearn.preprocessing import LabelEncoder
le=LabelEncoder()

In [ ]:
for i in list_cate:
    
    train_df[i]=le.fit_transform(train_df[i])

In [ ]:
for i in test_cate:
    df_test[i]=le.fit_transform(df_test[i])

In [ ]:
df_test

In [ ]:
train_df['cleaning_fee']=le.fit_transform(train_df['cleaning_fee'])

In [ ]:
df_test['cleaning_fee']=le.fit_transform(df_test['cleaning_fee'])

In [ ]:
x=train_df

In [ ]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,random_state=0,test_size=0.2)

In [ ]:
from sklearn.linear_model import LinearRegression
lr=LinearRegression()
lr.fit(x_train,y_train)

In [ ]:
pred_1=lr.predict(x_test)

In [ ]:
from sklearn.metrics import r2_score
score_1=r2_score(y_test,pred_1)

In [ ]:
score_1

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV


In [ ]:
rgb=GradientBoostingRegressor()
param_grid={'n_estimators':[100,500], 
            'learning_rate': [0.1,0.05,0.02],
            'max_depth':[4], 
            'min_samples_leaf':[3], 
            'max_features':[1.0] }
    

In [ ]:
clf=GridSearchCV(rgb,param_grid,cv=5,verbose=0)

In [ ]:
clf.fit(x_train,y_train)

In [ ]:
rgb=GradientBoostingRegressor(learning_rate=0.1,max_depth=4,max_features=1.0,n_estimators=500)

In [ ]:
rgb.fit(x_train,y_train)
pred_2=rgb.predict(x_test)

In [ ]:
score_2=r2_score(y_test,pred_2)

In [ ]:
score_2

In [ ]:
preds=rgb.predict(df_test)

In [ ]:
submission=pd.DataFrame({'id':Id,
                       'predictions':preds})

In [ ]:
submission